In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

import s3fs
import os
import torch
import random
import json
# import sktime

from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from redshift import get_from_redshift
from datetime import datetime
from config import Config  
from collections import defaultdict, Counter
from tqdm import tqdm 
from itertools import zip_longest

tqdm.pandas()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Loading data

In [11]:
%%writefile isco08.sql
SELECT * FROM dev.intermediate.isco08_unit_group

Writing isco08.sql


In [12]:
get_from_redshift("isco08.sql", Config.redshift_creds)

,repository_timestamp,code_isco08_unitgroup,omschrijving_isco08_unitgroup,batchid,batch_ts
0,2018-03-01 07:45:31.903342,2621,Archivists and curators,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
1,2018-03-01 07:45:31.903342,3352,Government tax and excise officials,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
2,2018-03-01 07:45:31.903342,8331,Bus and tram drivers,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
3,2018-03-01 07:45:31.903342,1330,Information and communications technology serv...,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
4,2018-03-01 07:45:31.903342,4321,Stock clerks,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
...,...,...,...,...,...
431,2018-03-01 07:45:31.903342,1411,Hotel managers,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
432,2018-03-01 07:45:31.903342,7318,"Handicraft workers in textile, leather and rel...",3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
433,2018-03-01 07:45:31.903342,7215,Riggers and cable splicers,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604
434,2018-03-01 07:45:31.903342,3333,Employment agents and contractors,3ebd0872-5648-421e-a9d3-fdbd0f4ec498,2017-07-14 11:15:50.626604


### Work experience

In [6]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_work_experience) 

In [7]:
candidate_work_experience = get_from_redshift(sql_path, Config.redshift_creds)
candidate_work_experience.head()

,candidate_id,unique_id,date_start_job,date_end_job,function_id,isco_functie_niveau,isco_code4,function_name_self,company_name,source
0,84556,197598,2000-08-01 00:00:00,2001-01-04,1236.0,2.0,5131.0,Functienaam van code 5322,(oud 3146Amsterdam RAI Catering Services,plaatsing
1,84556,197971,2000-09-07 00:00:00,2001-01-01,1065.0,1.0,9412.0,Functienaam van code 5319,(oud 3146Amsterdam RAI Catering Services,plaatsing
2,84556,193936,2000-09-30 00:00:00,2001-07-02,1236.0,2.0,5131.0,Functienaam van code 5328,AMSTERDAM RAI CATERING SERVICES B.V.,plaatsing
3,84556,191027,2000-10-23 00:00:00,2000-10-27,2038.0,1.0,9329.0,Functienaam van code 9970,Formido Bouwmarkt,plaatsing
4,84556,191793,2003-12-22 00:00:00,2004-01-02,2038.0,1.0,9329.0,Functienaam van code 9970,Yakult Nederland B.V.,plaatsing


### Education

In [5]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_education) 

In [6]:
candidate_education = get_from_redshift(sql_path, Config.redshift_creds)
candidate_education.head()

,candidate_id,education_level,date_start,date_end,date_registrated,passed
0,297604,2,1999-07-01 00:00:00,2000-07-01 00:00:00,2005-08-17 13:22:15,0
1,167633,2,2004-08-01 00:00:00,2005-03-01 00:00:00,2005-08-09 17:51:40,0
2,388845,1,1989-09-01 00:00:00,1993-06-01 00:00:00,2005-08-30 11:47:41,1
3,404474,2,1995-09-01 00:00:00,1996-07-01 00:00:00,2005-09-06 16:31:05,0
4,439908,2,1985-08-01 00:00:00,1989-06-01 00:00:00,2005-09-06 18:30:10,1


### Skills

In [128]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_skills) 

#### What's date start here? 

In [129]:
candidate_skill = get_from_redshift(sql_path, Config.redshift_creds)
candidate_skill.head()

,candidate_id,skill_id,date_start
0,1083185,3026,2010-03-09 18:37:30
1,2234015,41,2010-03-09 18:37:55
2,2340092,3065,2010-03-09 18:37:57
3,601383,13,2010-03-09 18:38:19
4,930604,2971,2010-03-09 18:38:24


### Languages

In [37]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_languages) 

In [38]:
candidate_languages = get_from_redshift(sql_path, Config.redshift_creds)
candidate_languages.head()

,candidate_id,language_id,date_start
0,347855,15,2010-03-09 18:37:14
1,1422075,15,2010-03-09 18:37:37
2,1464620,15,2010-03-09 18:37:39
3,2265249,15,2010-03-09 18:37:55
4,2774257,17,2010-03-09 18:38:09


In [39]:
lang_to_idx = {v : i for i, v in enumerate(sorted(candidate_languages["language_id"].astype(float).unique()))}
idx_to_lang = {v : i for i, v in lang_to_idx.items()}
candidate_languages["language_id"] = candidate_languages["language_id"].progress_apply(lambda x: lang_to_idx[x])

100%|██████████| 2109279/2109279 [00:04<00:00, 506904.40it/s]


In [41]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_mapping_languages) 

In [42]:
language_mappings = get_from_redshift(sql_path, Config.redshift_creds)
language_mappings.head()

,language_id,language
0,297,Deens
1,303,Japans
2,11,Berber
3,296,Zweeds
4,317,Pools


In [46]:
language_mapper = language_mappings.set_index("language_id").to_dict()["language"]

In [49]:
index_to_language = {k: language_mapper[v] for k, v in idx_to_lang.items()}

In [90]:
candidate_languages["language_id"] = candidate_languages["language_id"].apply(lambda x: index_to_language[x])

In [139]:
candidate_languages.drop("date_start", axis=1).to_csv("../Data/languages_mappings.csv")

In [91]:
languages_oh = pd.crosstab(candidate_languages["candidate_id"],
                           candidate_languages["language_id"])

In [92]:
languages_oh.head()

language_id,Arabisch,Berber,Chinees,Deens,Duits,Engels,Fins,Frans,Fries,Hongaars,...,Nederlands,Noors,Pools,Portugees,Roemeens,Russisch,Spaans,Tjechisch/Slowaaks,Turks,Zweeds
candidate_id,,,,,,,,,,,,,,,,,,,,,
84267,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
84349,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
84381,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
84386,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
84432,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [93]:
languages_oh.to_csv("../Data/languages_mappings.csv")

### Addresses

In [112]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_addresses) 

In [113]:
candidate_addresses = get_from_redshift(sql_path, Config.redshift_creds)
candidate_addresses.head()

,candidate_id,from_post_code,date_start,date_end
0,612659,8262 PD,2007-05-02 00:00:00,None
1,5534405,8225 RZ,2014-01-14 00:00:00,None
2,2845568,1774 AL,2010-12-10 00:00:00,2011-04-01 00:00:00
3,5024986,9249 NE,2013-07-25 00:00:00,2013-08-31 00:00:00
4,2361856,5264 PC,2013-10-15 00:00:00,None


In [114]:
candidate_addresses.shape

(4175920, 4)

In [115]:
# Drop nonsense rows
candidate_addresses = candidate_addresses[(candidate_addresses["date_start"] >= dt.datetime(1950, 1, 1)) &
                                          (candidate_addresses["date_start"] <= dt.datetime(2025, 1, 1))]

In [116]:
candidate_addresses.shape

(4175769, 4)

In [117]:
# Only store for candidates we will actually be using
candidate_addresses = candidate_addresses[candidate_addresses["candidate_id"].isin(
    set(candidate_work_experience["candidate_id"].unique()))]

In [118]:
candidate_addresses.shape

(660735, 4)

In [119]:
most_recent_address = candidate_addresses.groupby("candidate_id")["date_start"].progress_apply(max)

100%|██████████| 434925/434925 [01:57<00:00, 3699.66it/s]


In [120]:
# TODO: do this in a way that isn't hideous
makeshift_index = set((most_recent_address.index.astype(str) + most_recent_address.astype(str)).values)

In [121]:
candidate_addresses.loc[:,"temp"] = candidate_addresses["candidate_id"].astype(str) + candidate_addresses["date_start"].astype(str)

In [122]:
candidate_addresses = candidate_addresses[candidate_addresses["temp"].isin(makeshift_index)].drop("temp", axis=1)

In [123]:
candidate_addresses["from_post_code"] = candidate_addresses["from_post_code"].str.slice(0, 4)

In [124]:
candidate_final_addresses = candidate_addresses.groupby("candidate_id")["from_post_code"].progress_apply(lambda x: x.values[-1])

100%|██████████| 434925/434925 [00:12<00:00, 34501.32it/s]


In [127]:
candidate_final_addresses.to_csv("../Data/address_mappings.csv")

In [33]:
zip_to_idx = {v : i for i, v in enumerate(sorted(candidate_final_addresses.unique()))}
idx_to_zip = {v : i for i, v in zip_to_idx.items()}
candidate_final_addresses = candidate_final_addresses.progress_apply(lambda x: zip_to_idx[x])

100%|██████████| 530009/530009 [00:00<00:00, 541301.16it/s]


In [53]:
with open("../Data/index_to_zip_code.json", "w") as outfile:
    json.dump(idx_to_zip, outfile) 

In [35]:
addresses_one_hot = pd.get_dummies(candidate_final_addresses)

addresses_one_hot.head()

,0,1,2,3,4,5,6,7,8,9,...,4747,4748,4749,4750,4751,4752,4753,4754,4755,4756
candidate_id,,,,,,,,,,,,,,,,,,,,,
84556,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85627,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# DIY Sparse matrix
compressed_addresses_one_hot = addresses_one_hot.idxmax(axis=1)

In [38]:
compressed_addresses_one_hot.to_csv("../Data/addresses_one-hot.csv")

In [46]:
for k, v in compressed_addresses_one_hot.items():
    print(k, v)
    a = np.zeros(4757)
    a[v] = 1
    break

84556 90
[   0 3174 3173 ... 1587 4756   90]


In [54]:
emb = nn.Embedding(4757, 10)

In [60]:
emb(torch.LongTensor([90])), emb(torch.LongTensor([50]))

(tensor([[-0.7321,  0.4676,  0.0742, -1.5482,  1.1119, -0.9195, -1.2665, -0.5721,
           0.0527,  0.2775]], grad_fn=<EmbeddingBackward>),
 tensor([[ 0.9187,  0.3937, -0.8414,  0.1928, -0.3927,  0.9119, -0.7422,  0.6156,
           0.2376,  1.0105]], grad_fn=<EmbeddingBackward>))

### Driving licenses

In [102]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_driving_license) 

In [103]:
candidate_driving = get_from_redshift(sql_path, Config.redshift_creds)
candidate_driving.head()

,candidate_id,driving_licenses,date_driving_license
0,431387,A,2005-09-06 17:59:19
1,419193,A,2014-09-08 21:28:34
2,448864,A,2010-10-21 11:16:50
3,590700,A,2014-06-04 09:48:16
4,911212,A,2013-11-07 13:24:56


In [104]:
candidate_driving = candidate_driving[candidate_driving["candidate_id"].isin(set(candidate_work_experience["candidate_id"].unique()))]

In [106]:
candidate_driving[["candidate_id", "driving_licenses"]].set_index("candidate_id").to_csv("../Data/license_mappings.csv")

In [114]:
licenses_one_hot = pd.crosstab(candidate_driving["candidate_id"],
                               candidate_driving["driving_licenses"])

In [115]:
licenses_one_hot.shape

(216926, 8)

In [116]:
licenses_one_hot.to_csv("../Data/licenses_one-hot.csv")

# Skill reindexing

In [54]:
skill_to_idx = {v : i for i, v in enumerate(sorted(candidate_skill["skill_id"].astype(float).unique()))}
idx_to_skill = {v : i for i, v in skill_to_idx.items()}
candidate_skill["skill_id"] = candidate_skill["skill_id"].progress_apply(lambda x: skill_to_idx[x])
skills_dict = candidate_skill.groupby("candidate_id")["skill_id"].apply(list).to_dict()

100%|██████████| 4190460/4190460 [00:06<00:00, 658433.97it/s]


In [57]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_skills_mapping) 
skill_mappings = get_from_redshift(sql_path, Config.redshift_creds)
skill_mappings.head()

,skill_id,skill_name,vce_id
0,37,Magazijnwerkzaamheden,12
1,3224,Rijervaring winkeldistributie,12
2,2986,Novell: Besturingssystemen,7
3,2942,Dienblad lopen,6
4,3225,Rijervaring wipcar,12


In [59]:
skill_mappings = skill_mappings.set_index("skill_id")["skill_name"].to_dict()

In [61]:
index_to_skill = {k: skill_mappings[v] for k, v in idx_to_skill.items()}

In [95]:
candidate_skill["skill_id"] = candidate_skill["skill_id"].apply(lambda x: index_to_skill[x])

In [100]:
candidate_skill.to_csv("../Data/candidate_skills_mapped.csv")

In [ ]:
# Convert lists of skills to dataframe of skills per candidate
skills_ct = pd.crosstab(candidate_skill["candidate_id"], 
                        candidate_skill["skill_id"])

# # skills = skills_ct.where(skills_ct != 1, skills_ct.columns.to_series(), axis=1)
skills_ct.columns = [f"skill_{i}" for i in skills_ct.columns]

In [ ]:
skills_ct.head()

In [11]:
# skills_ct.to_csv("skills_one-hot.csv")

# Merging and formatting

In [63]:
def clean_df(df, table = "work_experience"):
    
    if table == "work_experience":
        start, end = "date_start_job", "date_end_job"
    elif table == "education":
        start, end = "date_start", "date_end"
    else:
        return NotImplemented
    
    # Drop nonsense data
    df = df[df[start] >= dt.datetime(1800, 1, 1)]
    df = df[df[end] <= dt.datetime(2100, 1, 1)]

    # Convert datetime to date
    df[start] = pd.to_datetime(df[start]).dt.date
    df[end] = pd.to_datetime(df[end]).dt.date
    
    return df
        
candidate_work_experience = clean_df(candidate_work_experience)
candidate_education = clean_df(candidate_education, table = "education")

In [64]:
candidate_work_experience["time_spent"] = (candidate_work_experience["date_end_job"] - 
                                           candidate_work_experience["date_start_job"]).dt.days.astype('int16')

In [65]:
def find_educations(df_work, df_education):

    # Merge career data with education levels
    career_education = pd.merge(df_work, 
                                df_education, 
                                on = "candidate_id", 
                                how = "left")[["candidate_id",
                                               "date_start_job",
                                               "date_end_job",
                                               "company_name",
                                               "time_spent",
                                               "source",
                                               "education_level",
                                               "date_start",
                                               "date_end",
                                               "passed"]]
    
    # Filter out education that were not passed (yet) at the time of starting a job
    passed = career_education[(career_education["date_start_job"] >= career_education["date_end"]) & 
                              (career_education["passed"] == 1)]
    
    # Only store the highest education level reached at the start of each job
    education_through_time = passed.groupby(["candidate_id", "date_start_job"])["education_level"].max()
    
    df_work.set_index(["candidate_id", "date_start_job"], inplace=True)

    # Store education data in candidate_work_experience
    df_work["education"] = education_through_time

    # np.nan education = no education
    df_work["education"].fillna(0, inplace=True)

    # Reset index for further data augmentation
    df_work.reset_index(inplace=True)
        
    return df_work

candidate_work_experience = find_educations(candidate_work_experience, candidate_education)

candidate_work_experience.head()

,candidate_id,date_start_job,unique_id,date_end_job,function_id,isco_functie_niveau,isco_code4,function_name_self,company_name,source,time_spent,education
0,92931,2018-10-01,17225527,2020-06-01,NaN,NaN,NaN,"Vrijwilliger Ontvangst, Behandelcentrum, Opname",Isala,dossier,609,3.0
1,92931,2019-12-01,17225530,2020-08-01,NaN,NaN,NaN,Receptioniste afdeling Sport & Bewegen\t,Landstede,dossier,244,3.0
2,92931,2020-01-01,17225534,2020-05-01,NaN,NaN,NaN,Receptioniste afdeling Sportcentrum,Landstede,dossier,121,3.0
3,93728,1995-02-01,14606857,1995-11-01,NaN,NaN,NaN,Sergeant lijnploeg,Ministerie van defensie,dossier,273,0.0
4,93728,2003-04-28,2207107,2003-05-02,1184.0,1.0,9333.0,Functienaam van code 9721,Friesland Foods Fresh** NIET GEBRUIKEN**,plaatsing,4,3.0


In [66]:
companies = dict(zip(set(candidate_work_experience["company_name"].values), range(len(candidate_work_experience["company_name"].unique()))))

In [69]:
index_to_company = {v: k for k, v in companies.items()}

with open("../Data/index_to_company.json", "w") as outfile:
    json.dump(index_to_company, outfile) 

In [55]:
candidate_work_experience["company_name"] = candidate_work_experience["company_name"].apply(lambda x: companies[x])

In [56]:
candidate_work_experience["source"] = candidate_work_experience["source"].apply(lambda x: {"plaatsing": 0, "dossier": 1}[x])

In [57]:
# Add skills
# candidate_work_experience = pd.merge(candidate_work_experience, 
#                                      skills_ct, 
#                                      left_on="candidate_id", 
#                                      right_index=True, 
#                                      how="left")

# Filtering and reindexing

In [70]:
more_than_5 = candidate_work_experience["isco_code4"].value_counts()
more_than_5 = set(more_than_5[more_than_5 > 5].index)
candidate_work_experience = candidate_work_experience[candidate_work_experience["isco_code4"].isin(more_than_5)]

In [71]:
id_to_idx = {v : i for i, v in enumerate(sorted(candidate_work_experience["function_id"].astype(float).unique()))}
idx_to_id = {v : i for i, v in id_to_idx.items()}

code_to_idx = {v : i for i, v in enumerate(sorted(candidate_work_experience["isco_code4"].astype(float).unique()))}
idx_to_code = {v : i for i, v in code_to_idx.items()}

In [88]:
with open("../Data/index_to_isco.json", "w") as outfile:
    json.dump(idx_to_code, outfile) 

In [74]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_mapping_function_groups) 
function_mappings = get_from_redshift(sql_path, Config.redshift_creds)
function_mappings.head()


,function_id,function_group
0,834,36
1,1167,39
2,2310,3150
3,2021,16
4,1503,19


In [60]:
candidate_work_experience["function_id"] = candidate_work_experience["function_id"].apply(lambda x: id_to_idx.get(x))
candidate_work_experience["isco_code4"] = candidate_work_experience["isco_code4"].apply(lambda x: code_to_idx[x])

In [61]:
candidate_work_experience.shape

(1667336, 12)

# Add CV embeddings

In [62]:
embedding_pd = pd.read_parquet("s3://s3-nl-prd-semrb-emr/embeddings/doc_embeddings/word2vec/word2vec_doc_embedding.parquet")
embedding_pd['cv_id'] = embedding_pd['cv_id'].astype('int')
embedding_pd.rename(columns={"doc_embedding": "tensor"}, inplace=True)

In [63]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_cvs) 

candidate_cvs = get_from_redshift(sql_path, Config.redshift_creds)

In [64]:
def find_embeddings(embedding_pd, candidate_cvs):
    embedding_pd = embedding_pd.set_index("cv_id")
    
    candidate_cvs = candidate_cvs[["cv_id", "candidate_id", "date_start"]]

    embedding_per_candidate = pd.merge(embedding_pd, 
                                       candidate_cvs, 
                                       left_index=True, 
                                       right_on="cv_id")[["candidate_id", "embedding", "date_start"]]
    
    matches = (set(embedding_per_candidate["candidate_id"]) & set(candidate_work_experience["candidate_id"]))
    
    matched_cvs = embedding_per_candidate[embedding_per_candidate["candidate_id"].isin(matches)]
    matched_cvs["date_start"] = matched_cvs["date_start"].dt.date
    
    last_cv_per_day = matched_cvs.groupby(["candidate_id", "date_start"])["embedding"].apply(lambda x: list(x)[-1])
    
    cv_embeddings = last_cv_per_day.reset_index()
    
    return cv_embeddings

cv_embeddings = find_embeddings(embedding_pd, candidate_cvs)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
cv_embeddings.head()

,candidate_id,date_start,embedding
0,84556.0,2020-07-02,"[0.07265971601009369, -0.8582056760787964, -0...."
1,84612.0,2020-05-30,"[0.3474675714969635, -1.0429900884628296, -0.3..."
2,84612.0,2020-06-03,"[0.34068500995635986, -1.0179638862609863, -0...."
3,84731.0,2020-06-03,"[0.12455234676599503, -0.1725415289402008, -0...."
4,85437.0,2020-06-06,"[0.2048000991344452, -0.49068811535835266, -0...."


In [66]:
def add_embeddings(candidate_work_experience, cv_embeddings):
    
    # Add CVs to candidates (includes duplicates)
    full_merge = pd.merge(candidate_work_experience,
                          cv_embeddings,
                          left_on="candidate_id",
                          right_on="candidate_id",
                          how="left")
    
    # display(full_merge)
    
    # Find the maximum date of uploaded cvs per job (i.e. most recent CV during each job)
    most_recent_cvs = full_merge[(full_merge["date_start_job"]
                                  >= full_merge["date_start"])].groupby(
        "unique_id")["date_start"].idxmax()   
    
    # Some candidates only added CVs after their last job started, so account for that too
    late_cvs = full_merge[(full_merge["date_start_job"] <= 
                           full_merge["date_start"])].groupby("unique_id")["date_start"].idxmax()
    
    # Filter out everything we already found earlier
    late_cvs = late_cvs[set(late_cvs.index) - set(most_recent_cvs.index)]
    
    # Now we have all the CVs in one place
    cv_idxs = pd.concat([most_recent_cvs, late_cvs])
    
    # Combine the two frames to include candidates without any CVs
    combined = pd.concat([full_merge[full_merge["embedding"].isna()], 
                          full_merge.loc[cv_idxs.values]])
    
    return combined

candidate_work_experience = add_embeddings(candidate_work_experience, cv_embeddings)

In [67]:
# candidate_work_experience["embedding"] = candidate_work_experience["embedding"].apply(lambda x: x if x is not np.nan
#                                                                                                   else [0] * 300)

In [68]:
candidate_work_experience = candidate_work_experience.reset_index().drop("index", axis=1)

In [69]:
# embedding_lists = candidate_work_experience["embedding"].to_list()
# embeddings = pd.DataFrame(embedding_lists)

In [70]:
# embeddings.columns = [f"embedding_{i}" for i in range(len(embeddings.columns))]

In [71]:
# candidate_work_experience = pd.concat([candidate_work_experience, embeddings], axis=1)

# Ordering & Time

In [72]:
def add_order(df):
    
    # Count the number of jobs each candidate has ahd
    job_counts = df.groupby("candidate_id").size()  
    
    # Sort by candidate_id, date_start_job
    sorted_df = df.sort_values(by = ['candidate_id', "date_start_job"])
    
    # Reset index
    sorted_df.reset_index(inplace=True, drop=True)
    
    # Create a list of lists containing the order of each candidates jobs (which came first, second, third, etc.)
    order = [np.arange(count) for count in job_counts.values]
    
    # Flatten list
    order = [item for sublist in order for item in sublist]
    
    # Add order to df
    sorted_df["job_order"] = order
    
    # Set a candidate_id, job_order as the index
    return sorted_df.set_index(["candidate_id", 
                                "job_order"])

df = add_order(candidate_work_experience)

In [73]:
df = df[["date_start_job", "date_end_job", "time_spent", "isco_code4", "function_id",
         "isco_functie_niveau", "company_name", "source", "education", "embedding"]]

df.head(10)

date_start_job date_end_job  time_spent  isco_code4  \
candidate_id job_order                                                       
84556        0             2000-08-01   2001-01-04         156         208   
             1             2000-09-07   2001-01-01         116         348   
             2             2000-09-30   2001-07-02         275         208   
             3             2000-10-23   2000-10-27           4         344   
             4             2003-12-22   2004-01-02          11         344   
             5             2004-07-12   2004-07-13           1         344   
             6             2005-04-11   2005-07-01          81         344   
             7             2005-05-25   2005-06-01           7         344   
             8             2005-07-27   2005-08-26          30         344   
             9             2006-03-16   2006-05-15          60         345   

                        function_id  isco_functie_niveau  company_name  \
candidate_id job_order                                                   
84556        0                  936                  2.0        324258   
             1                  809                  1.0        324258   
             2                  936                  2.0        329244   
             3                 1519                  1.0        368140   
             4                 1519                  1.0        419826   
             5                 1519                  1.0        419826   
             6                 1519                  1.0        206450   
             7                 1519                  1.0        231269   
             8                 1519                  1.0         54594   
             9                  893                  1.0        171163   

                        source  education  \
candidate_id job_order                      
84556        0               0        0.0   
             1               0        0.0   
             2               0        0.0   
             3               0        0.0   
             4               0        0.0   
             5               0        0.0   
             6               0        0.0   
             7               0        0.0   
             8               0        0.0   
             9               0        0.0   

                                                                embedding  
candidate_id job_order                                                     
84556        0          [0.07265971601009369, -0.8582056760787964, -0....  
             1          [0.07265971601009369, -0.8582056760787964, -0....  
             2          [0.07265971601009369, -0.8582056760787964, -0....  
             3          [0.07265971601009369, -0.8582056760787964, -0....  
             4          [0.07265971601009369, -0.8582056760787964, -0....  
             5          [0.07265971601009369, -0.8582056760787964, -0....  
             6          [0.07265971601009369, -0.8582056760787964, -0....  
             7          [0.07265971601009369, -0.8582056760787964, -0....  
             8          [0.07265971601009369, -0.8582056760787964, -0....  
             9          [0.07265971601009369, -0.8582056760787964, -0....

In [74]:
def change_steps(df):
    """Groups users' careers by same job types. I.e., if someone didn't
       change the type of job they had, but just the company, it will be
       grouped into one time step."""
    
    new_values = defaultdict(list)
    
    total_time_spent = 0
    education = 0
    date_end_job = dt.date(1970, 1, 1)
    
    for i in tqdm(range(len(df))):
        if i < len(df) - 1:
            current = df.iloc[i]
            next_ = df.iloc[i + 1]
            # Group all consecutive time steps with the same job 
            if (current.name[0] != next_.name[0]) or (current["isco_code4"] != next_["isco_code4"]):
                # Add values of last correct row
                total_time_spent += current["time_spent"]
                if current["education"] > education:
                    education = current["education"]

                if current["date_end_job"] > date_end_job:
                    date_end_job = current["date_end_job"]

                # Store results
                new_values["candidate_id"].append(current.name[0])
                new_values["date_start_job"].append(current["date_start_job"])
                new_values["date_end_job"].append(date_end_job)
                new_values["time_spent"].append(total_time_spent)
                new_values["isco_code4"].append(current["isco_code4"])
                new_values["function_id"].append(current["function_id"])
                new_values["isco_functie_niveau"].append(current["isco_functie_niveau"])
                new_values["education"].append(education)
                new_values["embedding"].append(current["embedding"])

                # Reset values
                total_time_spent = 0
                education = 0
                date_end_job = dt.date(1970, 1, 1)           
            else:
                total_time_spent += current["time_spent"]
                if current["education"] > education:
                    education = current["education"]

                if current["date_end_job"] > date_end_job:
                    date_end_job = current["date_end_job"]

    return pd.DataFrame(new_values)

# df = change_steps(df)

In [75]:
df.head()

date_start_job date_end_job  time_spent  isco_code4  \
candidate_id job_order                                                       
84556        0             2000-08-01   2001-01-04         156         208   
             1             2000-09-07   2001-01-01         116         348   
             2             2000-09-30   2001-07-02         275         208   
             3             2000-10-23   2000-10-27           4         344   
             4             2003-12-22   2004-01-02          11         344   

                        function_id  isco_functie_niveau  company_name  \
candidate_id job_order                                                   
84556        0                  936                  2.0        324258   
             1                  809                  1.0        324258   
             2                  936                  2.0        329244   
             3                 1519                  1.0        368140   
             4                 1519                  1.0        419826   

                        source  education  \
candidate_id job_order                      
84556        0               0        0.0   
             1               0        0.0   
             2               0        0.0   
             3               0        0.0   
             4               0        0.0   

                                                                embedding  
candidate_id job_order                                                     
84556        0          [0.07265971601009369, -0.8582056760787964, -0....  
             1          [0.07265971601009369, -0.8582056760787964, -0....  
             2          [0.07265971601009369, -0.8582056760787964, -0....  
             3          [0.07265971601009369, -0.8582056760787964, -0....  
             4          [0.07265971601009369, -0.8582056760787964, -0....

In [76]:
# Calculate the time it took to go from one job to another (in order)
df["time_between"] = df.groupby(
    "candidate_id")["date_start_job"].progress_apply(lambda x: x - x.shift(1))

100%|██████████| 469568/469568 [07:08<00:00, 1095.55it/s]


In [77]:
df["time_between"] = df["time_between"].shift(-1).fillna(pd.Timedelta(seconds=0)).dt.days.astype('int16')

In [78]:
df.head()

date_start_job date_end_job  time_spent  isco_code4  \
candidate_id job_order                                                       
84556        0             2000-08-01   2001-01-04         156         208   
             1             2000-09-07   2001-01-01         116         348   
             2             2000-09-30   2001-07-02         275         208   
             3             2000-10-23   2000-10-27           4         344   
             4             2003-12-22   2004-01-02          11         344   

                        function_id  isco_functie_niveau  company_name  \
candidate_id job_order                                                   
84556        0                  936                  2.0        324258   
             1                  809                  1.0        324258   
             2                  936                  2.0        329244   
             3                 1519                  1.0        368140   
             4                 1519                  1.0        419826   

                        source  education  \
candidate_id job_order                      
84556        0               0        0.0   
             1               0        0.0   
             2               0        0.0   
             3               0        0.0   
             4               0        0.0   

                                                                embedding  \
candidate_id job_order                                                      
84556        0          [0.07265971601009369, -0.8582056760787964, -0....   
             1          [0.07265971601009369, -0.8582056760787964, -0....   
             2          [0.07265971601009369, -0.8582056760787964, -0....   
             3          [0.07265971601009369, -0.8582056760787964, -0....   
             4          [0.07265971601009369, -0.8582056760787964, -0....   

                        time_between  
candidate_id job_order                
84556        0                    37  
             1                    23  
             2                    23  
             3                  1155  
             4                   203

In [79]:
num_classes = len(df["isco_code4"].unique())

In [81]:
# df_pred = df[["isco_functie_niveau", "education", "function_id"]].fillna(0)
pred_cols = ["time_between", "time_spent", "isco_functie_niveau", 
             "source", "education", "company_name", "function_id", "isco_code4"] # + [col for col in df.columns if "skill_" in col]
num_features = len(pred_cols)

df_pred = df[pred_cols].fillna(0)

In [82]:
df_pred.head()

time_between  time_spent  isco_functie_niveau  source  \
candidate_id job_order                                                          
84556        0                    37         156                  2.0       0   
             1                    23         116                  1.0       0   
             2                    23         275                  2.0       0   
             3                  1155           4                  1.0       0   
             4                   203          11                  1.0       0   

                        education  company_name  function_id  isco_code4  
candidate_id job_order                                                    
84556        0                0.0        324258          936         208  
             1                0.0        324258          809         348  
             2                0.0        329244          936         208  
             3                0.0        368140         1519         344  
             4                0.0        419826         1519         344

In [41]:
df_pred[["function_id", "isco_code4"]].reset_index().to_csv("../Data/DTW_df_pred2.csv")

### Create separate embedding dict

Done for optimization purposes

In [51]:
embedding_order = df.reset_index()[["candidate_id", "embedding"]]

In [52]:
grouped_embeddings = embedding_order.groupby("candidate_id")

In [53]:
embedding_a = defaultdict(lambda: defaultdict(list))

# Find each time step at which the candidate got a new CV
for candidate, values in tqdm(grouped_embeddings):
    embeddings = values["embedding"].values
  
    if (type(embeddings[0]) == type(np.array([]))) and np.nan not in embeddings[0]:
        embedding_a[candidate][0] = embeddings[0]
        
        for i, embedding in enumerate(embeddings):
            if i > 0:
                truths = embeddings[i - 1] != embedding

                if (type(truths) == type(np.array([]))) and (truths).all():
                    embedding_a[candidate][i] = embedding

100%|██████████| 469073/469073 [01:21<00:00, 5736.13it/s]


In [54]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [55]:
with open('../Data/embeddings_grouped.json', 'w') as f:
    json.dump(embedding_a, f, cls=NumpyEncoder)

In [84]:
df_pred.reset_index().to_csv("../Data/df_pred_ext.csv")

In [ ]:
pd.Series(skills_dict).to_csv("../Data/skills.csv")

### Candidate Certificates

In [130]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_candidate_certificates) 

candidate_certificates = get_from_redshift(sql_path, Config.redshift_creds)
candidate_certificates.head()

,candidate_id,candidate_certificate_id,date_start
0,466181,16,2005-09-13 17:30:13
1,517627,16,2005-11-22 16:57:02
2,1220503,25,2006-07-14 15:17:58
3,1320318,25,2006-10-25 09:55:50
4,804362,47,2006-11-23 16:07:17


In [131]:
sql_path = os.path.join("/home/ec2-user/SageMaker/data-science-development/talent_recommender/daily_snapshots",
                        Config.query_mapping_certificates) 

certificate_mappings = get_from_redshift(sql_path, Config.redshift_creds)
certificate_mappings = certificate_mappings.set_index("request_certificate_ids").to_dict()["request_certificate_description"]

In [132]:
candidate_certificates["candidate_certificate_id"] = candidate_certificates["candidate_certificate_id"].apply(lambda x: certificate_mappings[x] if x in certificate_mappings else x)

In [136]:
candidate_certificates.drop(["date_start"], axis=1).to_csv("../Data/candidate_certificates_mappings.csv")

In [84]:
candidate_certificates = pd.crosstab(candidate_certificates["candidate_id"], 
                                     candidate_certificates["candidate_certificate_id"])
candidate_certificates.head()

candidate_certificate_id,8,ADR Basis,ADR klasse 1+7,ATP vervoer bederfelijke goederen/geconditioneerd transport,Ambulanten training,Autolaadkraan,BHV,BIG - registratie,BWT 1,BWT 2 (ABW2),...,Wft PEplus schadeverzekeringen particulier,Wft PEplus schadeverzekeringen zakelijk,Wft PEplus vermogen,Wft Pensioenen,Wft Schadeverzekeringen (oud),"Wft Schadeverzekeringen Bedrijven (oud),",Wft Schadeverzekeringen Particulier,Wft Schadeverzekeringen Zakelijk,Wft Vermogen,ZW
candidate_id,,,,,,,,,,,,,,,,,,,,,
84603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85214,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
candidate_certificates.to_csv("../Data/candidate_certificates_mappings.csv")

### CV Embeddings

In [ ]:
df_pred = pd.read_csv("../Data/df_pred.csv")
skills = pd.read_csv("../Data/skills.csv")

In [ ]:
# cvPath = 's3a://s3-nl-prd-datahub-projects/smartmatch_cv/parsed_cv'

# s3 = s3fs.S3FileSystem()
# parsedCv_pd = pq.ParquetDataset(cvPath, filesystem=s3).read_pandas().to_pandas() 

In [ ]:
# s3 = s3fs.S3FileSystem()

# embedding_pd = pq.ParquetDataset('s3a://s3-nl-prd-semrb-emr/embeddings/doc_embeddings/xlm-roberta-base-smartmatch', filesystem=s3).read_pandas().to_pandas()

# embedding_pd['cv_id'] = embedding_pd['cv_id'].astype('int')
# embedding_pd.rename(columns={"doc_embedding": "tensor"}, inplace=True)
# embedding_pd.head() 

In [ ]:
# parsedCv_pd

In [ ]:
len(set(cv_embeddings["candidate_id"]) & set(df_pred["candidate_id"]))

In [ ]:
cv_embeddings.to_csv("../Data/cv_w2v_embeddings.csv")